In [1]:
import sys
from pathlib import Path

import numpy as np
from astropy import table, nddata
from astropy.io import fits
from scipy import signal, optimize
from matplotlib import pyplot as plt
from matplotlib import gridspec, colors
import cmocean
import betterplotlib as bpl

bpl.set_style()

# need to add the correct path to import utils
legus_home_dir = Path(".").resolve().parent
sys.path.append(str(legus_home_dir / "pipeline"))
import utils
import fit_utils

Other parameters

In [2]:
oversampling_factor = 2
psf_size = 15
snapshot_size = 30
snapshot_size_oversampled = snapshot_size * oversampling_factor

## Then we can load the data

In [3]:
def load_data(galaxy_name):
    data_dir = legus_home_dir / "data" / galaxy_name
    image_data, _, _ = utils.get_drc_image(data_dir)

    error_data = fits.open(data_dir / "size" / "sigma_electrons.fits")["PRIMARY"].data
    mask_data = fits.open(data_dir / "size" / "mask_image.fits")["PRIMARY"].data

    psf_name = f"psf_my_stars_{psf_size}_pixels_{oversampling_factor}x_oversampled.fits"
    psf = fits.open(data_dir / "size" / psf_name)["PRIMARY"].data
    psf_cen = int((psf.shape[1] - 1.0) / 2.0)
    # the convolution requires the psf to be normalized, and without any negative values
    psf = np.maximum(psf, 0)
    psf /= np.sum(psf)

    cat_name_ryon = (
        f"final_catalog_final_{snapshot_size}_pixels_psf_"
        f"my_stars_{psf_size}_pixels_{oversampling_factor}x_oversampled_ryonlike.txt"
    )
    cat_name_mine = (
        f"final_catalog_final_{snapshot_size}_pixels_psf_"
        f"my_stars_{psf_size}_pixels_{oversampling_factor}x_oversampled.txt"
    )
    cat_ryon = table.Table.read(str(data_dir / "size" / cat_name_ryon), format="ascii.ecsv")
    cat_mine = table.Table.read(str(data_dir / "size" / cat_name_mine), format="ascii.ecsv")
    
    return image_data, error_data, mask_data, psf, cat_ryon, cat_mine

## Convenience functions to be used later

In [4]:
def create_radial_profile(model_psf_bin_image, cluster_snapshot, mask, x_c, y_c):
    """
    Make a radial profile of cluster and model pixel values

    :param model_psf_bin_image: the model image on the same pixel scale as the data
    :param cluster_snapshot: the data snapshot
    :param mask: the mask indicating which pixel values to not use
    :param x_c: X coordinate of the center in snapshot coordinates
    :param y_c: Y coordinate of the center in snapshot coordinates
    :return: Three numpy arrays: The radii of all pixel values, in sorted order, the
             model values at these radii, then the data values at these radii
    """
    # When fitting I treated the center of the pixels as the integer location, so do
    # that here too
    radii, model_ys, data_ys = [], [], []
    for x in range(model_psf_bin_image.shape[1]):
        for y in range(model_psf_bin_image.shape[0]):
            if mask[y][x] > 0:
                radii.append(fit_utils.distance(x, y, x_c, y_c))
                model_ys.append(model_psf_bin_image[y, x])
                data_ys.append(cluster_snapshot[y, x])

    # sort everything in order of radii
    idxs = np.argsort(radii)
    return np.array(radii)[idxs], np.array(model_ys)[idxs], np.array(data_ys)[idxs]

def bin_profile(radii, pixel_values, bin_size):
    """
    Take an existing profile and bin it azimuthally

    :param radii: Radii values corresponding to the pixel values
    :param pixel_values: Values at the radii passed in
    :param bin_size: How big the bins should be, in pixels
    :return: Binned radii and pixel values
    """
    binned_radii, binned_ys = [], []
    for r_min in np.arange(0, int(np.ceil(max(radii))), bin_size):
        r_max = r_min + bin_size
        idx_above = np.where(r_min < radii)
        idx_below = np.where(r_max > radii)
        idx_good = np.intersect1d(idx_above, idx_below)

        if len(idx_good) > 0:
            binned_radii.append(r_min + 0.5 * bin_size)
            binned_ys.append(np.mean(pixel_values[idx_good]))

    return np.array(binned_radii), np.array(binned_ys)

In [5]:
def plot_model_set(
    cluster_snapshot,
    uncertainty_snapshot,
    mask,
    psf,
    params_ryon,
    params_mine,
    r_eff_ryon,
    r_eff_mine,
    galaxy,
    cluster_id,
    save_loc
):
    params1 = params_ryon
    params2 = params_mine
    
    # the center is in oversampled coords, get the regular image center
    x_c = fit_utils.oversampled_to_image(params1[1], oversampling_factor)
    y_c = fit_utils.oversampled_to_image(params1[2], oversampling_factor)
    
    # create copies of the image
    cluster_snapshot = cluster_snapshot.copy()
    uncertainty_snapshot = uncertainty_snapshot.copy()
    mask = mask.copy()
    # do the radial weighting. Need to get the data coordinates of the center
    weight_snapshot = fit_utils.radial_weighting(
        uncertainty_snapshot,
        fit_utils.oversampled_to_image(params1[1], oversampling_factor),
        fit_utils.oversampled_to_image(params1[2], oversampling_factor),
        style="annulus",
    )
    


    model_image1, model_psf_image1, model_psf_bin_image1 = fit_utils.create_model_image(
        *params1, psf, snapshot_size_oversampled, oversampling_factor
    )

    model_image2, model_psf_image2, model_psf_bin_image2 = fit_utils.create_model_image(
        *params2, psf, snapshot_size_oversampled, oversampling_factor
    )

    diff_image1 = cluster_snapshot - model_psf_bin_image1
    diff_image2 = cluster_snapshot - model_psf_bin_image2
    sigma_image1 = weight_snapshot * diff_image1 / uncertainty_snapshot
    sigma_image2 = weight_snapshot * diff_image2 / uncertainty_snapshot
    
    # have zeros in the sigma image where the mask has zeros, but leave it unmodified
    # otherwise
    sigma_image1 *= np.minimum(mask, 1.0)
    sigma_image2 *= np.minimum(mask, 1.0)

    # set up the normalizations and colormaps
    # Use the data image to get the normalization that will be used in all plots. Base
    # it on the data so that it is the same in all bootstrap iterations
    vmax = 2 * np.max(cluster_snapshot)
    linthresh = 3 * np.min(uncertainty_snapshot)
    data_norm = colors.SymLogNorm(
        vmin=-vmax, vmax=vmax, linthresh=linthresh, base=10
    )
    sigma_norm = colors.Normalize(vmin=-10, vmax=10)
    u_norm = colors.Normalize(0, vmax=1.2 * np.max(uncertainty_snapshot))
    m_norm = colors.Normalize(0, vmax=np.max(mask))

    data_cmap = bpl.cm.lisbon
    sigma_cmap = cmocean.cm.tarn  # "bwr_r" also works
    u_cmap = cmocean.cm.deep_r
    m_cmap = cmocean.cm.gray_r

    # create the figure and add all the subplots
    fig = plt.figure(figsize=[25, 15])
    gs = gridspec.GridSpec(
        nrows=6,
        ncols=5,
        width_ratios=[10, 10, 10, 1, 15],  # have a dummy spacer column
        wspace=0.1,
        hspace=0.7,
        left=0.01,
        right=0.98,
        bottom=0.06,
        top=0.94,
    )
    ax_d = fig.add_subplot(gs[0:2, 0], projection="bpl")  # data
    ax_u = fig.add_subplot(gs[2:4, 0], projection="bpl")  # uncertainty
    ax_m = fig.add_subplot(gs[4:, 0], projection="bpl")  # mask
    
    ax_r1 = fig.add_subplot(gs[0:2, 1], projection="bpl")  # raw model
    ax_f1 = fig.add_subplot(gs[2:4, 1], projection="bpl")  # full model (f for fit)
    ax_s1 = fig.add_subplot(gs[4:, 1], projection="bpl")  # sigma difference
    
    ax_r2 = fig.add_subplot(gs[0:2, 2], projection="bpl")  # raw model
    ax_f2 = fig.add_subplot(gs[2:4, 2], projection="bpl")  # full model (f for fit)
    ax_s2 = fig.add_subplot(gs[4:, 2], projection="bpl")  # sigma difference
    
    
    ax_pd = fig.add_subplot(
        gs[0:3, 4], projection="bpl"
    )  # radial profile differential
    ax_pc = fig.add_subplot(
        gs[3:, 4], projection="bpl"
    )  # radial profile cumulative

    # show the images in their respective panels
    common_data = {"norm": data_norm, "cmap": data_cmap}
    d_im = ax_d.imshow(cluster_snapshot, **common_data, origin="lower")
    m_im = ax_m.imshow(mask_snapshot, norm=m_norm, cmap=m_cmap, origin="lower")
    u_im = ax_u.imshow(uncertainty_snapshot, norm=u_norm, cmap=u_cmap, origin="lower")
    
    r_im1 = ax_r1.imshow(model_image1, **common_data, origin="lower")
    f_im1 = ax_f1.imshow(model_psf_bin_image1, **common_data, origin="lower")
    s_im1 = ax_s1.imshow(sigma_image1, norm=sigma_norm, cmap=sigma_cmap, origin="lower")
    r_im2 = ax_r2.imshow(model_image2, **common_data, origin="lower")
    f_im2 = ax_f2.imshow(model_psf_bin_image2, **common_data, origin="lower")
    s_im2 = ax_s2.imshow(sigma_image2, norm=sigma_norm, cmap=sigma_cmap, origin="lower")

    fig.colorbar(d_im, ax=ax_d)
    fig.colorbar(m_im, ax=ax_m)
    fig.colorbar(u_im, ax=ax_u)
    fig.colorbar(r_im1, ax=ax_r1)
    fig.colorbar(f_im1, ax=ax_f1)
    fig.colorbar(s_im1, ax=ax_s1)
    fig.colorbar(r_im2, ax=ax_r2)
    fig.colorbar(f_im2, ax=ax_f2)
    fig.colorbar(s_im2, ax=ax_s2)

    ax_d.set_title("Artificial Cluster Data")
    ax_m.set_title("Mask")
    ax_u.set_title("Uncertainty")
    
    ax_r1.set_title("Ryon-like\nRaw Cluster Model")
    ax_f1.set_title("Model Convolved\nwith PSF and Binned")
    ax_s1.set_title("(Data - Model)/Uncertainty")
    
    ax_r2.set_title("Fit\nRaw Cluster Model")
    ax_f2.set_title("Model Convolved\nwith PSF and Binned")
    ax_s2.set_title("(Data - Model)/Uncertainty")

    for ax in [ax_d, ax_m, ax_u, ax_r1, ax_r2, ax_f1, ax_f1, ax_s1, ax_s2]:
        ax.remove_labels("both")
        ax.remove_spines(["all"])

    # Then make the radial plots. first background subtract
    cluster_snapshot -= params1[7]
    model_psf_bin_image1 -= params1[7]
    model_psf_bin_image2 -= params2[7]

    c_d = bpl.color_cycle[0]
    c_m1 = bpl.color_cycle[1]
    c_m2 = bpl.color_cycle[2]

    radii, model_ys1, data_ys = create_radial_profile(
        model_psf_bin_image1, cluster_snapshot, mask, x_c, y_c
    )
    radii, model_ys2, data_ys = create_radial_profile(
        model_psf_bin_image2, cluster_snapshot, mask, x_c, y_c
    )

    ax_pd.scatter(radii, data_ys, c=c_d, s=5, alpha=1.0, label="Data")
    ax_pd.scatter(radii, model_ys1, c=c_m1, s=5, alpha=1.0, label="Ryon-like")
    ax_pd.scatter(radii, model_ys2, c=c_m2, s=5, alpha=1.0, label="Full Method")
    ax_pd.axhline(0, ls=":", c=bpl.almost_black) 

    # then bin this data to make the binned plot
    ax_pd.plot(*bin_profile(radii, data_ys, 1.0), c=c_d, lw=5, label="Binned Data")
    ax_pd.plot(
        *bin_profile(radii, model_ys1, 1.0), c=c_m1, lw=5, label="Binned Ryon-like"
    )
    ax_pd.plot(
        *bin_profile(radii, model_ys2, 1.0), c=c_m2, lw=5, label="Binned Full"
    )

    ax_pd.legend(loc="upper right")
    ax_pd.add_labels(
        "Radius (pixels)", "Pixel Value [$e^{-}$]", f"{galaxy} {cluster_id}"
    )
    # set min and max values so it's easier to flip through bootstrapping plots
    y_min = np.min(cluster_snapshot)
    y_max = np.max(cluster_snapshot)
    # give them a bit of padding
    diff = y_max - y_min
    y_min -= 0.1 * diff
    y_max += 0.1 * diff
    ax_pd.set_limits(0, np.ceil(max(radii)), y_min, y_max)

    # then make the cumulative one. The radii are already in order so this is easy
    model_ys_cumulative1 = np.cumsum(model_ys1)
    model_ys_cumulative2 = np.cumsum(model_ys2)
    data_ys_cumulative = np.cumsum(data_ys)

    ax_pc.plot(radii, data_ys_cumulative, c=c_d, label="Data")
    ax_pc.plot(radii, model_ys_cumulative1, c=c_m1, label="Ryon-like")
    ax_pc.plot(radii, model_ys_cumulative2, c=c_m2, label="Full Method")
    ax_pc.set_limits(0, np.ceil(max(radii)), 0, 1.2 * np.max(data_ys_cumulative))
    ax_pc.legend(loc="upper left")
    ax_pc.add_labels(
        "Radius (pixels)", "Cumulative Pixel Values [$e^{-}$]"
    )

    # the last one just has the list of parameters
    ax_pc.easy_add_text(
        "Param = Ryon-like - Full Method\n"
        f"scale radius [pixels] = {params1[3]:.2g} - {params2[3]:.2g}\n"
        f"q (axis ratio) = {params1[4]:.2f} - {params2[4]:.2f}\n"
        f"position angle = {params1[5]:.2f} - {params2[5]:.2f}\n"
        f"$\eta$ (power law slope) = {params1[6]:.2f} - {params2[6]:.2f}\n"
        f"background = {params1[7]:.2f} - {params2[7]:.2f}\n"
        f"$\chi^2$ = {np.sum(np.abs(sigma_image1)):,.4f} - {np.sum(np.abs(sigma_image2)):,.4f}\n\n"
        "$R_{eff}$ [pixels]" + f" = {r_eff_ryon:.2f} - {r_eff_mine:.2f}\n",
        "lower right",
        fontsize=15,
    )
    fig.savefig(save_loc)
    plt.close(fig)
    del fig

In [6]:
# Then plot the guesses

for gal in ["ngc628-c", "ngc628-e", "ngc1313-e", "ngc1313-w"]:
    image_data, error_data, mask_data, psf, cat_ryon, cat_mine = load_data(gal)
    # then find the correct row
    for row_r, row_m in zip(cat_ryon, cat_mine):
        r_eff_ryon = row_r["r_eff_pixels_rmax_15pix_best"]
        r_eff_mine = row_m['r_eff_pixels_rmax_15pix_best']
        
        # only look at the different ones
        if (
            abs(r_eff_ryon - r_eff_mine) / r_eff_mine > 0.5
            and row_r["power_law_slope_best"] > 1.3
            and row_r["good_radius"]
            and row_m["good_radius"]
        ):
            print(f"{gal:9} {row_m['ID']:4}: full={r_eff_mine:.2f}, Ryon-like={r_eff_ryon:.2f}")
        else:
            continue

        # create the snapshot. We use ceiling to get the integer pixel values as python
        # indexing does not include the final value. So when we calcualte the offset, it
        # naturally gets biased low. Moving the center up fixes that in the easiest way.
        x_cen = int(np.ceil(row_m["x_fitted_best"]))
        y_cen = int(np.ceil(row_m["y_fitted_best"]))

        # Get the snapshot, based on the size desired.
        # Since we took the ceil of the center, go more in the negative direction (i.e.
        # use ceil to get the minimum values). This only matters if the snapshot size is odd
        x_min = x_cen - int(np.ceil(snapshot_size / 2.0))
        x_max = x_cen + int(np.floor(snapshot_size / 2.0))
        y_min = y_cen - int(np.ceil(snapshot_size / 2.0))
        y_max = y_cen + int(np.floor(snapshot_size / 2.0))

        data_snapshot = image_data[y_min:y_max, x_min:x_max].copy()
        error_snapshot = error_data[y_min:y_max, x_min:x_max].copy()
        mask_snapshot = mask_data[y_min:y_max, x_min:x_max].copy()

        snapshot_x_cen_fit_r = row_r["x_fitted_best"] - x_min
        snapshot_y_cen_fit_r = row_r["y_fitted_best"] - y_min
        snapshot_x_cen_fit_m = row_m["x_fitted_best"] - x_min
        snapshot_y_cen_fit_m = row_m["y_fitted_best"] - y_min
        
        # Use the same mask region as was used in the actual fitting procedure
        mask_snapshot = fit_utils.handle_mask(mask_snapshot, row_m["ID"])

        ryon_params = [
            row_r["log_luminosity_best"],
            fit_utils.image_to_oversampled(snapshot_x_cen_fit_r, oversampling_factor),
            fit_utils.image_to_oversampled(snapshot_y_cen_fit_r, oversampling_factor),
            row_r["scale_radius_pixels_best"],
            row_r["axis_ratio_best"],
            row_r["position_angle_best"],
            row_r["power_law_slope_best"],
            row_r["local_background_best"],
        ]
        mine_params = [
            row_m["log_luminosity_best"],
            fit_utils.image_to_oversampled(snapshot_x_cen_fit_m, oversampling_factor),
            fit_utils.image_to_oversampled(snapshot_y_cen_fit_m, oversampling_factor),
            row_m["scale_radius_pixels_best"],
            row_m["axis_ratio_best"],
            row_m["position_angle_best"],
            row_m["power_law_slope_best"],
            row_m["local_background_best"],
        ]

        plot_model_set(data_snapshot, error_snapshot, mask_snapshot, psf,
                       ryon_params, 
                       mine_params, 
                       r_eff_ryon, 
                       r_eff_mine,
                       gal, row_r["ID"],
                       f"../data/{gal}/size/cluster_fit_plots/fit_check_{gal}_{row_r['ID']}.png")
print("done")

ngc628-c   265: full=1.65, Ryon-like=2.57
ngc628-c   378: full=3.12, Ryon-like=4.70
ngc628-c   428: full=2.25, Ryon-like=4.64
ngc628-c   815: full=2.70, Ryon-like=4.27
ngc628-c   961: full=1.77, Ryon-like=2.77
ngc628-c  1181: full=2.62, Ryon-like=1.29
ngc628-c  2007: full=1.72, Ryon-like=2.64
ngc628-c  2039: full=3.18, Ryon-like=5.05
ngc628-c  2748: full=2.33, Ryon-like=3.80
ngc628-c  2842: full=3.64, Ryon-like=5.54
ngc628-c  2986: full=0.52, Ryon-like=0.25
ngc628-c  2987: full=1.70, Ryon-like=2.71
ngc628-c  3017: full=0.63, Ryon-like=0.31
ngc628-e    79: full=3.17, Ryon-like=5.65
ngc628-e   267: full=1.39, Ryon-like=2.10
ngc1313-e  108: full=5.15, Ryon-like=9.07
ngc1313-e  213: full=5.30, Ryon-like=9.32
ngc1313-e  428: full=3.26, Ryon-like=6.44
ngc1313-e  665: full=1.05, Ryon-like=1.88
ngc1313-e 1855: full=3.51, Ryon-like=5.85
ngc1313-w  236: full=3.28, Ryon-like=5.18
ngc1313-w 1740: full=4.93, Ryon-like=8.00
ngc1313-w 2267: full=6.98, Ryon-like=13.66
ngc1313-w 2366: full=2.61, Ryon-l